In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster
import mysql.connector
from mysql.connector import Error

# Configurações do banco de dados
DB_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "12344",
    "database": "mercado_imobiliario"
}

# Função para conectar ao banco de dados
def conectar_banco():
    try:
        conexao = mysql.connector.connect(**DB_CONFIG)
        return conexao
    except Error as e:
        print(f"Erro ao conectar ao banco: {e}")
        return None

# Função para listar tabelas disponíveis
def listar_tabelas():
    conexao = conectar_banco()
    if conexao:
        cursor = conexao.cursor()
        cursor.execute("SHOW TABLES;")
        tabelas = [tabela[0] for tabela in cursor.fetchall()]
        conexao.close()
        return tabelas
    return []

# Função para carregar os dados do banco
def carregar_dados():
    tabelas = listar_tabelas()
    if not tabelas:
        print("Nenhuma tabela encontrada no banco de dados.")
        return None
    
    tabela = tabelas[0]  # Carregar a primeira tabela disponível
    print(f"Carregando dados da tabela: {tabela}")
    conexao = conectar_banco()
    if conexao:
        query = f"SELECT * FROM `{tabela}`;"
        df = pd.read_sql(query, conexao)
        conexao.close()
        return df
    else:
        return None

# Carregar os dados
df = carregar_dados()
if df is None or df.empty:
    print("Erro ao carregar dados do banco ou DataFrame vazio.")
else:
    # Ajustar nomes de colunas
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("-", "_")

    # Exibir primeiras linhas
    display(df.head())

    # Informações gerais do dataset
    df.info()

    # Estatísticas descritivas
    display(df.describe())

    # Distribuição de preços
    if 'preco' in df.columns and not df['preco'].isnull().all():
        plt.figure(figsize=(10, 5))
        sns.histplot(df['preco'].dropna(), bins=50, kde=True)
        plt.xlabel('Preço dos Imóveis')
        plt.ylabel('Frequência')
        plt.title('Distribuição de Preços dos Imóveis')
        plt.show()
    
    # Correlação entre variáveis numéricas
    df_numeric = df.select_dtypes(include=['number'])
    if not df_numeric.empty:
        plt.figure(figsize=(10, 6))
        sns.heatmap(df_numeric.corr(), annot=True, cmap='coolwarm', fmt='.2f')
        plt.title("Matriz de Correlação")
        plt.show()
    else:
        print("Nenhuma coluna numérica disponível para calcular a matriz de correlação.")

    # Mapa Geoespacial com Folium
    if 'latitude' in df.columns and 'longitude' in df.columns and not df[['latitude', 'longitude']].isnull().all().all():
        mapa = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=10)
        marker_cluster = MarkerCluster().add_to(mapa)
        
        for _, row in df.dropna(subset=['latitude', 'longitude']).iterrows():
            folium.Marker(
                location=[row['latitude'], row['longitude']],
                popup=f"Preço: R${row['preco']:.2f}" if 'preco' in df.columns else "Localização"
            ).add_to(marker_cluster)
        
        mapa.save("mapa_imoveis.html")
        print("Mapa salvo como mapa_imoveis.html")

    # Boxplot de preços por estado
    if 'estado' in df.columns and 'preco' in df.columns and not df[['estado', 'preco']].isnull().all().all():
        plt.figure(figsize=(12, 6))
        sns.boxplot(x='estado', y='preco', data=df.dropna(subset=['estado', 'preco']))
        plt.xticks(rotation=45)
        plt.title("Distribuição de Preços por Estado")
        plt.show()
    else:
        print("Não há dados suficientes para gerar o boxplot de preços por estado.")


Carregando dados da tabela: sample_submission


,id,saleprice
0,1461.0,169277.0524984
1,1462.0,187758.39398876802
2,1463.0,183583.683569555
3,1464.0,179317.47751083
4,1465.0,150730.079976501


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         1459 non-null   object
 1   saleprice  1459 non-null   object
dtypes: object(2)
memory usage: 22.9+ KB


,id,saleprice
count,1459,1459
unique,1459,1438
top,2356.0,174388.867432503
freq,1,3


Nenhuma coluna numérica disponível para calcular a matriz de correlação.
Não há dados suficientes para gerar o boxplot de preços por estado.
